输入数据格式：
```
{"words": ["痛", "1", "天", "。"], "ner": ["B-SIGNS", "O", "O", "O"]}

```

In [2]:
%pip install modelscope
%pip install simplejson
%pip install datasets==2.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00


In [4]:
%pip install adaseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached

In [6]:
%pip install oss2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.8/283.8 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.1/443.1 kB 43.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oss2: filename=oss2-2.18.6-py3-none-any.whl size=118355 sha256=c149abd124022c178de7340f70f7e39ca2a7b9a6ad8a5e901e4b7a650ad65f64
  Stored in directory: /root/.cache/pip/wheels/e9/1c/df/6256a3d22097f6e1a30edd892de172054fd27875e0a349b4a4
  Created wheel for aliyun-python-sdk-core: filename=aliyun_python_sdk_core-2.15.1-py3-none-any.whl size=535325 sha256=0e4fd5e222ab08e24613cb3a2f0c2a9cd6d7b7ac6b9a003c961b133cf69de109
  Stored in directory: /root/.ca

In [7]:
from modelscope.utils.constant import Tasks
from modelscope.pipelines import pipeline

import json
import re
import os

模型采用：https://modelscope.cn/models/iic/nlp_raner_named-entity-recognition_chinese-large-generic

In [8]:
unlabled_corpus = open('./corpus/tsc_pcc_texts.txt', 'r').read().split('。')

In [9]:
unlabled_corpus[0]

'第篇概论本篇主要介绍计算机系统的基本组成应用与发展，并通过对本书结构的介绍，指出学习本书的基本思路'

In [10]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [34]:
class PCCDataset(Dataset):
  def __init__(self, corpus_list):
    self.corpus = corpus_list

  def __len__(self):
    return len(self.corpus)

  def __getitem__(self, index):
    return self.corpus[index]

dataset = PCCDataset(unlabled_corpus)
dataloader = DataLoader(dataset, batch_size=64, drop_last=True)

In [22]:
ner_pipeline = pipeline(Tasks.named_entity_recognition, 'damo/nlp_raner_named-entity-recognition_chinese-large-generic', batch_size=64)

2024-07-09 04:47:21,392 - modelscope - WARNING - Model revision not specified, use revision: v1.0.1
2024-07-09 04:47:21,768 - modelscope - INFO - initiate model from /root/.cache/modelscope/hub/damo/nlp_raner_named-entity-recognition_chinese-large-generic
2024-07-09 04:47:21,770 - modelscope - INFO - initiate model from location /root/.cache/modelscope/hub/damo/nlp_raner_named-entity-recognition_chinese-large-generic.
2024-07-09 04:47:21,772 - modelscope - INFO - initialize model from /root/.cache/modelscope/hub/damo/nlp_raner_named-entity-recognition_chinese-large-generic
2024-07-09 04:47:30,839 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
2024-07-09 04:47:33,359 - modelscope - INFO - All model checkpoint weights were used when initializing ModelForTokenClassificationWithCRF.

2024-07-09 04:47:33,362 - modelscope - INFO - All the weights of ModelForTokenClassificationWithCRF were initialized from the model checkpoint If your task is similar to the task the model o

In [43]:
results = {}

In [37]:
for i, batch in enumerate(dataloader):
  print(len(batch))

64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64


In [42]:
len(dataloader)

239

In [44]:
for i, batch in enumerate(dataloader):
  try:
    results[i] = ner_pipeline(batch)
  except:
    results[i] = {}

In [47]:
results

{0: [{'output': [{'type': 'CW',
     'start': 0,
     'end': 4,
     'prob': 0.11052113,
     'span': '第篇概论'}]},
  {'output': [{'type': 'CW',
     'start': 0,
     'end': 9,
     'prob': 0.12956186,
     'span': '第章计算机系统概论'}]},
  {'output': [{'type': 'CW',
     'start': 0,
     'end': 5,
     'prob': 0.11793805,
     'span': '计算机系统'},
    {'type': 'CW',
     'start': 16,
     'end': 21,
     'prob': 0.25201312,
     'span': '计算机系统'}]},
  {'output': [{'type': 'PROD',
     'start': 2,
     'end': 4,
     'prob': 0.2648286,
     'span': '硬件'}]},
  {'output': [{'type': 'CW',
     'start': 2,
     'end': 4,
     'prob': 0.26184863,
     'span': '软件'}]},
  {'output': [{'type': 'PROD',
     'start': 15,
     'end': 19,
     'prob': 0.1458124,
     'span': '磁带磁盘'}]},
  {'output': []},
  {'output': [{'type': 'CW',
     'start': 3,
     'end': 5,
     'prob': 0.2550694,
     'span': '软件'}]},
  {'output': [{'type': 'PROD',
     'start': 19,
     'end': 22,
     'prob': 0.16549416,
     'span': '软

In [50]:
with open('results.txt', 'w') as f:
  f.write(str(results))